# CIFAR-10 Sparse Reconstruction: Score-Based Neural Field

## Overview

**Approach**: Score-based diffusion with Perceiver IO + Fourier features

**Task**: Predict 20% output pixels from different 20% input pixels

**Architecture**:
```
Input: 204 input pixels (coords + RGB)
Output: 204 different pixels (coords only)
       ↓
Score Network s_θ(x_t, coords, t | input)
       ↓
Predicted Score: ∇_x log p_t(x | input)
       ↓
Langevin Sampling → Clean RGB predictions
```

## Theory: Score-Based Generative Modeling

### Forward SDE (Add Noise)
$$dx = -\frac{\beta_t}{2} x dt + \sqrt{\beta_t} dw$$

where $w$ is Brownian motion.

### Reverse SDE (Remove Noise)
$$dx = \left[-\frac{\beta_t}{2} x - \beta_t \nabla_x \log p_t(x)\right] dt + \sqrt{\beta_t} dw$$

### Score Function
Learn $s_\theta(x, t) \approx \nabla_x \log p_t(x)$ (gradient of log-density)

### Training Objective (Denoising Score Matching)
$$\mathcal{L} = \mathbb{E}_{t, x_0, \epsilon} \left[\lambda(t) \left\| s_\theta(x_t, t) + \frac{\epsilon}{\sigma_t} \right\|^2 \right]$$

where $x_t = \alpha_t x_0 + \sigma_t \epsilon$, $\epsilon \sim \mathcal{N}(0, I)$

### Why Score-Based?
- ✅ Principled probabilistic framework
- ✅ Flexible noise schedules
- ✅ Well-studied theory (Song et al., 2021)
- ⚠️ Langevin sampling slower than ODE solvers

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

# Import shared components
from core.neural_fields.perceiver import PerceiverIO, FourierFeatures
from core.sparse.cifar10_sparse import SparseCIFAR10Dataset
from core.sparse.metrics import MetricsTracker, print_metrics, visualize_predictions

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Noise Schedule

Variance Preserving (VP) SDE with cosine schedule

In [ ]:
def cosine_beta_schedule(timesteps, s=0.008):
    """Cosine schedule from Improved DDPM"""
    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0.0001, 0.9999)


class NoiseSchedule:
    """VP-SDE noise schedule"""
    def __init__(self, timesteps=1000, beta_schedule='cosine'):
        self.timesteps = timesteps
        
        if beta_schedule == 'cosine':
            betas = cosine_beta_schedule(timesteps)
        else:
            betas = torch.linspace(0.0001, 0.02, timesteps)
        
        alphas = 1.0 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        
        self.betas = betas
        self.alphas = alphas
        self.alphas_cumprod = alphas_cumprod
        self.sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod)


# Visualize schedule
schedule = NoiseSchedule(timesteps=1000)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(schedule.betas.numpy())
plt.title('β_t (noise variance)')
plt.xlabel('Timestep')
plt.grid(alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(schedule.alphas_cumprod.numpy())
plt.title('ᾱ_t (signal retention)')
plt.xlabel('Timestep')
plt.grid(alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(schedule.sqrt_one_minus_alphas_cumprod.numpy(), label='σ_t (noise std)')
plt.plot(schedule.sqrt_alphas_cumprod.numpy(), label='α_t (signal std)')
plt.title('Signal vs Noise')
plt.xlabel('Timestep')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Score Network Architecture

Perceiver IO backbone + time conditioning

In [ ]:
class SinusoidalTimeEmbedding(nn.Module):
    """Sinusoidal time embedding"""
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    
    def forward(self, t):
        """t: (B,) continuous timesteps in [0, T]"""
        device = t.device
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
        emb = t[:, None] * emb[None, :]
        emb = torch.cat([emb.sin(), emb.cos()], dim=-1)
        return emb


class ScoreNetwork(nn.Module):
    """
    Score network: s_θ(x_t, coords, t | input) → score
    
    Predicts ∇_x log p_t(x | input) for output pixel values
    """
    def __init__(
        self,
        num_latents=512,
        latent_dim=512,
        num_fourier_feats=256,
        num_blocks=6,
        num_heads=8,
        dropout=0.1
    ):
        super().__init__()
        
        # Perceiver IO backbone
        self.perceiver = PerceiverIO(
            input_channels=3,
            output_channels=3,
            num_latents=num_latents,
            latent_dim=latent_dim,
            num_fourier_feats=num_fourier_feats,
            num_blocks=num_blocks,
            num_heads=num_heads,
            dropout=dropout
        )
        
        # Time embedding
        self.time_embed = SinusoidalTimeEmbedding(latent_dim)
        self.time_mlp = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.SiLU(),
            nn.Linear(latent_dim, latent_dim)
        )
    
    def forward(self, noisy_values, output_coords, t, input_coords, input_values):
        """
        Args:
            noisy_values: (B, N_out, 3) noisy output pixel values
            output_coords: (B, N_out, 2) output pixel coordinates
            t: (B,) continuous timesteps
            input_coords: (B, N_in, 2) input pixel coordinates
            input_values: (B, N_in, 3) input pixel values (clean)
        
        Returns:
            score: (B, N_out, 3) predicted score ∇_x log p_t(x | input)
        """
        B = noisy_values.shape[0]
        
        # Time embedding
        t_emb = self.time_mlp(self.time_embed(t))  # (B, latent_dim)
        
        # Concatenate input (clean) and output (noisy) pixels
        all_coords = torch.cat([input_coords, output_coords], dim=1)  # (B, N_in+N_out, 2)
        
        # Add time embedding to pixel values
        input_values_t = input_values + t_emb.unsqueeze(1).expand(-1, input_values.shape[1], -1)[:, :, :3]
        noisy_values_t = noisy_values + t_emb.unsqueeze(1).expand(-1, noisy_values.shape[1], -1)[:, :, :3]
        
        all_values = torch.cat([input_values_t, noisy_values_t], dim=1)  # (B, N_in+N_out, 3)
        
        # Predict score at output coordinates
        score = self.perceiver(all_coords, all_values, output_coords)
        
        return score


# Test score network
model = ScoreNetwork().to(device)
test_noisy = torch.randn(4, 204, 3).to(device)
test_output_coords = torch.rand(4, 204, 2).to(device)
test_t = torch.rand(4).to(device) * 1000
test_input_coords = torch.rand(4, 204, 2).to(device)
test_input_values = torch.rand(4, 204, 3).to(device)

test_score = model(test_noisy, test_output_coords, test_t, test_input_coords, test_input_values)
print(f"Score network test: {test_score.shape}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 3. Training: Denoising Score Matching

In [ ]:
def train_score_matching(
    model,
    train_loader,
    schedule,
    epochs=100,
    lr=1e-4,
    device='cuda'
):
    """Train score network with denoising score matching"""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    losses = []
    
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_coords = batch['input_coords'].to(device)
            input_values = batch['input_values'].to(device)
            output_coords = batch['output_coords'].to(device)
            output_values = batch['output_values'].to(device)  # x_0 (clean)
            
            B = input_coords.shape[0]
            
            # Sample random timestep
            t = torch.randint(0, schedule.timesteps, (B,), device=device).float()
            
            # Add noise: x_t = α_t * x_0 + σ_t * ε
            noise = torch.randn_like(output_values)
            sqrt_alpha_t = schedule.sqrt_alphas_cumprod[t.long()].view(B, 1, 1).to(device)
            sqrt_sigma_t = schedule.sqrt_one_minus_alphas_cumprod[t.long()].view(B, 1, 1).to(device)
            
            noisy_values = sqrt_alpha_t * output_values + sqrt_sigma_t * noise
            
            # Predict score
            pred_score = model(noisy_values, output_coords, t, input_coords, input_values)
            
            # Target score: -ε / σ_t
            target_score = -noise / sqrt_sigma_t
            
            # Loss: ||predicted_score - target_score||^2
            loss = F.mse_loss(pred_score, target_score)
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        losses.append(avg_loss)
        scheduler.step()
        
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.6f}, LR = {scheduler.get_last_lr()[0]:.6f}")
        
        # Save checkpoint every 10 epochs
        if (epoch + 1) % 10 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
            }, f'score_network_epoch_{epoch+1}.pt')
    
    return losses


# Create dataset and dataloader
print("Loading CIFAR-10...")
train_dataset = SparseCIFAR10Dataset(
    root='../data',
    train=True,
    input_ratio=0.2,
    output_ratio=0.2,
    download=True,
    seed=42
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

print(f"Dataset size: {len(train_dataset)}")
print(f"Batches: {len(train_loader)}")

# Initialize model and schedule
model = ScoreNetwork(
    num_latents=512,
    latent_dim=512,
    num_fourier_feats=256,
    num_blocks=6,
    num_heads=8
).to(device)

schedule = NoiseSchedule(timesteps=1000, beta_schedule='cosine')

# Train
print("\nStarting training...")
losses = train_score_matching(model, train_loader, schedule, epochs=100, lr=1e-4, device=device)

## 4. Sampling: Langevin Dynamics

In [ ]:
@torch.no_grad()
def langevin_sample(
    model,
    schedule,
    output_coords,
    input_coords,
    input_values,
    num_steps=1000,
    device='cuda'
):
    """
    Sample using Langevin dynamics (reverse SDE)
    
    Args:
        model: Trained score network
        schedule: Noise schedule
        output_coords: (B, N_out, 2) coordinates to predict
        input_coords: (B, N_in, 2) input coordinates
        input_values: (B, N_in, 3) input values (conditioning)
        num_steps: Number of sampling steps
    
    Returns:
        x_0: (B, N_out, 3) predicted clean values
    """
    B = output_coords.shape[0]
    N_out = output_coords.shape[1]
    
    # Start from pure noise
    x_t = torch.randn(B, N_out, 3, device=device)
    
    # Langevin dynamics
    timesteps = torch.linspace(schedule.timesteps - 1, 0, num_steps).long()
    
    for i, t_idx in enumerate(tqdm(timesteps, desc="Sampling")):
        t = torch.full((B,), t_idx.item(), device=device, dtype=torch.float)
        
        # Predict score
        score = model(x_t, output_coords, t, input_coords, input_values)
        
        # Get noise schedule params
        beta_t = schedule.betas[t_idx].to(device)
        sqrt_beta_t = torch.sqrt(beta_t)
        
        # Langevin update with score
        if i < len(timesteps) - 1:
            noise = torch.randn_like(x_t)
            x_t = x_t + beta_t * score + sqrt_beta_t * noise
        else:
            # Last step: no noise
            x_t = x_t + beta_t * score
    
    return torch.clamp(x_t, 0, 1)

## 5. Evaluation with Unified Metrics

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Score Matching Loss')
plt.title('Training Loss: Score-Based Neural Field')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Evaluate on test set
print("\nEvaluating on test set...")
test_dataset = SparseCIFAR10Dataset(
    root='../data',
    train=False,
    input_ratio=0.2,
    output_ratio=0.2,
    download=True,
    seed=42
)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

model.eval()
tracker = MetricsTracker()

for batch in tqdm(test_loader, desc="Testing"):
    input_coords = batch['input_coords'].to(device)
    input_values = batch['input_values'].to(device)
    output_coords = batch['output_coords'].to(device)
    output_values = batch['output_values'].to(device)
    
    # Sample predictions
    pred_values = langevin_sample(
        model, schedule, output_coords, input_coords, input_values,
        num_steps=100, device=device
    )
    
    tracker.update(pred_values, output_values)

# Print results
results = tracker.compute()
print("\n" + "="*50)
print("SCORE-BASED NEURAL FIELD - Test Results")
print("="*50)
print_metrics(results)
print("="*50)

## 6. Visualize Predictions

In [ ]:
# Visualize predictions
sample_batch = next(iter(test_loader))
input_coords = sample_batch['input_coords'].to(device)
input_values = sample_batch['input_values'].to(device)
output_coords = sample_batch['output_coords'].to(device)
output_values = sample_batch['output_values'].to(device)
full_images = sample_batch['full_image'].to(device)

# Generate predictions
pred_values = langevin_sample(
    model, schedule, output_coords, input_coords, input_values,
    num_steps=100, device=device
)

# Visualize
fig = visualize_predictions(
    input_coords, input_values,
    output_coords, pred_values, output_values,
    full_images, n_samples=4
)
plt.suptitle('Score-Based Neural Field: Predictions vs Ground Truth', fontsize=14, y=1.02)
plt.savefig('score_based_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary

### ✅ Implemented
- Score-based diffusion with Perceiver IO
- Fourier feature positional encoding
- Denoising score matching training
- Langevin dynamics sampling

### 📊 Results
See metrics above for:
- MSE/MAE on output pixels
- PSNR/SSIM on full images

### ⚖️ Strengths & Weaknesses

**Strengths**:
- Principled probabilistic framework
- Flexible for uncertainty quantification
- Well-studied theory

**Weaknesses**:
- Langevin sampling slower (100-1000 steps)
- Requires careful tuning of step sizes
- More complex than direct denoising

### 🔄 Next
Compare with:
- Notebook 4: NF Denoiser (Gaussian RF)
- Notebook 5: Flow Matching